In [1]:
from Lib.myModules.my_ML_Modules_1 import *
import math
import sys
import contextlib

In [2]:
data_dir = "../data"
infantNum = 2
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')

ECG DATA: {'fs': 500, 'sig_len': 78916000, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['II'], 'comments': []}
RESP DATA: {'fs': 50, 'sig_len': 7891433, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
157832.0 sec, 2630.5333333333333 minutes, 43.84222222222222 hours
157828.66 sec, 2630.4776666666667 minutes, 43.84129444444444 hours


In [3]:
# initialize the training
start_time_secs = hours2secs(2)
end_time_secs = hours2secs(2.2)
lr_model, pr_model, svr_rbf_model = training_pipeline(start_time_secs=start_time_secs, 
                                                        end_time_secs=end_time_secs, 
                                                        signal_ecg_0=signal_ecg_0,
                                                        signal_resp_0=signal_resp_0,
                                                        ECG_dataset=ECG_dataset,
                                                        RESP_dataset=RESP_dataset)

start: 3600000, end: 3960000
start: 360000, end: 396000
90th percentile: 0.05979828971976473, 10th percentile: -0.09829855844344888, IQR: 0.15809684816321362
Identified outliers: 23525
Data Shape: (360000,)
90th percentile: 24.97982803434372, 10th percentile: 22.14494392558524, IQR: 2.83488410875848
Identified outliers: 492
Data Shape: (36000,)
[[188.58788423   1.70980388]
 [  1.70980388  91.70667357]]
Pearsons correlation: 0.013
Spearmans correlation: -0.077
---------------------------------------------
--- Linear Regression ---
Coefficient of determination: 0.00014198578194979383
Intercept: [122.613014]
Coefficients: [[0.00823566]]
Mean absolute error: 6.15
Mean squared error: 96.34
Root mean squared error: 9.82
---------------------------------------------
--- Polynomial Regression ---
Coefficient of determination: 0.015371050598505742
Intercept: [122.65165]
Coefficients: [[ 6.3478367e-10 -1.1062037e-08  6.1231242e-07  9.9342124e-06
  -4.3642780e-07  4.6474558e-09]]
Mean absolute er

In [4]:
step = 0.25
startHrs = 10
endHrs = 12
logFilePath = f"../logs/infant{infantNum}_step{step}_startHrs{startHrs}_endHrs{endHrs}.log"

with open(logFilePath, 'a') as f:
    with contextlib.redirect_stdout(f):
        for i in np.arange(startHrs, endHrs, step):
            print("-------------------------------------------------------------------------")
            print(f"Data bin index: {i:.3f}")
            start_time_secs = hours2secs(i)
            end_time_secs = hours2secs(i+step)
            lr_model, pr_model, svr_rbf_model = cascade_training_pipeline(lr_model=lr_model, 
                                                                            pr_model=pr_model, 
                                                                            svr_rbf_model=svr_rbf_model, 
                                                                            start_time_secs=start_time_secs, 
                                                                            end_time_secs=end_time_secs,
                                                                            signal_ecg_0=signal_ecg_0,
                                                                            signal_resp_0=signal_resp_0,
                                                                            ECG_dataset=ECG_dataset,
                                                                            RESP_dataset=RESP_dataset)

In [5]:
# save the model
for i in [lr_model, pr_model, svr_rbf_model]:
    filename = f"../models/infant_{infantNum}/{namestr(i, globals())[0]}_step{step}_startHrs{startHrs}_endHrs{endHrs}.sav"
    joblib.dump(i, filename)
